In [1]:
import os
os.chdir("/home/studio-lab-user/sagemaker-studiolab-notebooks/AI-OT24/Reinforcement-Learning-Stock-Porfolio-Managment")
!pip install -q -r requirements.txt
import sys
sys.path.append('./utils')
from trading_functions import *
# Core Libraries
import os
import sys
import time
import datetime
import warnings
#warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all messages, 1 = info, 2 = warnings, 3 = errors

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

# Financial Data
import yfinance as yf
import quantstats as qs
import ta

# Machine Learning - Supervised Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

# Machine Learning - Deep Learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.losses import BinaryCrossentropy

# Reinforcement Learning and Environments
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import A2C, DDPG, DQN, HER, PPO, SAC, TD3
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import (
    EvalCallback, StopTrainingOnRewardThreshold, StopTrainingOnNoModelImprovement
)
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecCheckNan
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from sb3_contrib import ARS, MaskablePPO, RecurrentPPO, QRDQN, TRPO

# Imitation Learning
from imitation.algorithms import bc
from imitation.testing.reward_improvement import is_significant_reward_improvement
from imitation.data.types import Transitions

# Interactive Brokers API
from ib_insync import *

from typing import Callable

from collections import Counter

clear_output()


In [ ]:
# Define constants
SEED = 1
history_length = 5
reward_type = 'LNR'
stocks = ['AAPL', 'AMZN', 'META', 'MSFT', 'NVDA', 'TSLA']
start_date = '2020-01-01'
end_date = '2023-06-30'
date_splits = 4
n_envs = 10
n_steps = 8
total_timesteps = 10_000
batch_size = 16
learning_rate = 0.001
ent_coef = 0.10
log_interval = 1_000
eval_freq=1_000

current_date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_save_dir = f'./models/model_{current_date}'
log_dir = f'./logs/log_{current_date}'

# Training environment
env, vec_env = create_training_env(history_length, reward_type, '2020-01-01', '2023-06-30', stocks, n_envs)

# Validation environment
valid_env, vec_valid_env = create_evaluation_env(history_length, reward_type, '2023-07-01', '2023-12-30', stocks)

# Test environment
test_env, _ = create_evaluation_env(history_length, reward_type, '2024-01-01', '2024-10-30', stocks, n_envs=1)

# Evaluation callback for saving the best model
eval_callback = EvalCallback(
    vec_valid_env,
    n_eval_episodes=1,
    eval_freq=eval_freq,
    deterministic=True,
    verbose=1,
    best_model_save_path=model_save_dir,
)

import gymnasium as gym
import torch as th

from stable_baselines3 import PPO

# Custom actor (pi) and value function (vf) networks
# of two layers of size 32 each with Relu activation function
# Note: an extra linear layer will be added on top of the pi and the vf nets, respectively
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=dict(pi=[64, 64], vf=[64, 64]))

model = PPO("MlpPolicy", vec_env, device="cpu", tensorboard_log = log_dir, learning_rate = 0.02, policy_kwargs=policy_kwargs)
model.learn(total_timesteps=100_000, callback = eval_callback, log_interval=10, tb_log_name='PPO', reset_num_timesteps=True, progress_bar=True)
vec_env.close()
date = None
date_length = env.df_unscaled.shape[0]
for i in range(date_splits):
    date_length = date_length//2
    print(date_length,date)
    date = env.df_unscaled.index[-date_length]
    _, vec_env = create_training_env(history_length, reward_type, date, end_date, stocks, n_envs)
    model.set_env(vec_env)
    param = model.get_parameters()
    param['policy.optimizer']['param_groups'][0]['lr'] = param['policy.optimizer']['param_groups'][0]['lr']/2
    model.set_parameters(param)
    model.learn(total_timesteps=100_000, callback = eval_callback, log_interval=10, tb_log_name='PPO', reset_num_timesteps=False, progress_bar=True)
    vec_env.close()
    
# Evaluate the model
evaluate_model(env, model, 'Train', SEED, has_policy=False)
evaluate_model(valid_env, model, 'Valid', SEED, has_policy=False)
evaluate_model(test_env, model, 'Test', SEED, has_policy=False)


# Close environments
env.close()
valid_env.close()
test_env.close()
vec_valid_env.close()

  19% ━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18,990/100,000  [ 0:00:17 < 0:01:15 , 1,087 it/s ]

In [3]:
"""

# Evaluation callback for saving the best model
eval_callback = EvalCallback(
    vec_valid_env,
    n_eval_episodes=1,
    eval_freq=eval_freq,
    deterministic=True,
    verbose=1,
    best_model_save_path=model_save_dir,
)
stop_train_callback = StopTrainingOnNoModelImprovement(max_no_improvement_evals=3, min_evals=5, verbose=1)


# Train the model
model = train_model(
    model_name="PPO",  # Can be 'PPO', 'A2C', or 'DQN'
    create_model=True,
    vec_env=vec_env,
    iterations=1,
    train_timesteps=total_timesteps,
    log_frec=log_interval,
    log_base_dir=log_dir,
    n_steps=n_steps,
    batch_size=batch_size,
    learning_rate=learning_rate,
    ent_coef=ent_coef
)
"""

'\n\n# Evaluation callback for saving the best model\neval_callback = EvalCallback(\n    vec_valid_env,\n    n_eval_episodes=1,\n    eval_freq=eval_freq,\n    deterministic=True,\n    verbose=1,\n    best_model_save_path=model_save_dir,\n)\nstop_train_callback = StopTrainingOnNoModelImprovement(max_no_improvement_evals=3, min_evals=5, verbose=1)\n\n\n# Train the model\nmodel = train_model(\n    model_name="PPO",  # Can be \'PPO\', \'A2C\', or \'DQN\'\n    create_model=True,\n    vec_env=vec_env,\n    iterations=1,\n    train_timesteps=total_timesteps,\n    log_frec=log_interval,\n    log_base_dir=log_dir,\n    n_steps=n_steps,\n    batch_size=batch_size,\n    learning_rate=learning_rate,\n    ent_coef=ent_coef\n)\n'

In [35]:
param = model.get_parameters()
print(param['policy.optimizer']['param_groups'][0]['lr'])
param['policy.optimizer']['param_groups'][0]['lr'] = param['policy.optimizer']['param_groups'][0]['lr']/2
print(param['policy.optimizer']['param_groups'][0]['lr'])
model.set_parameters(param)
param = model.get_parameters()
print(param['policy.optimizer']['param_groups'][0]['lr'])

0.0003
0.00015
0.00015
